In [1]:
import pandas as pd 
import tensorflow as tf
from matplotlib import pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

#Import dataset
training_df = pd.read_csv(filepath_or_buffer="https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")

#scale the value
training_df['median_house_value'] /= 1000.0

#Print the firt value of the dataset
print(training_df.head())
print(training_df.describe())


#implement model 

def build_model(my_learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=1, input_shape = (1,)))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=my_learning_rate),
                  loss = "mean_squared_error",
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model
# code for train model
def train_model(model, df, feature, label, epochs, batch_size):
    #feed and train model by giving feature and label
    # the model will train for a specified number of epochs
    history = model.fit(x=df[feature],
                        y=df[label],
                        batch_size=batch_size,
                        epochs = epochs)
    #gather the train model's weight and bias
    trained_weight = model.get_weights()[0][0]
    trained_bias = model.get_weights()[1]

    epochs = history.epoch
    hist = pd.DataFrame(history.history)

    #track progression of training: we take snapeshot of model's root mean squared error at each epochs
    rmse = hist["root_mean_squared_error"]
    
    return trained_weight, trained_bias, epochs, rmse
print("Defined the build_model and train_model functions.")

#define the plotting function 
def plot_the_model(train_weight, trained_bias, feature, label):

    # Label the axes
    plt.xlabel(feature)
    plt.ylabel(label)

    #create a scatter plot 
    random_examples = training_df.sample(n=200)
    plt.scatter(random_examples[feature], random_examples[label])

    # create red line for the prediction of model
    x0 = 0
    y0 = trained_bias
    x1 = random_examples[feature].max()
    y1 = trained_bias + (train_weight * x1)
    plt.plot([x0,x1], [y0, y1], c = 'r')

    plt.show()
    
# plot of the loss curve 
def plot_the_loss_curve(epochs, rmse):
    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Root Mean Squared Error")

    plt.plot(epochs, rmse, label = "Loss")
    plt.legend()
    plt.ylim([rmse.min()* 0.97, rmse.max()])
    plt.show()
print("Defined the plot_the_model and plot_the_loss_curve functions.")

# hyperparameters 
training_df["room_per_person"] = training_df["total_rooms"]/training_df["population"]

learning_rate = 0.07
epochs = 20
batch_size = 35

# specifying the feature and label
my_feature = "median_income"
my_label = "median_house_value"

#discard old model
my_model = None

# invoke new model
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df,
                                         my_feature, my_label,
                                         epochs, batch_size)
print("\nThe learning rate for the model is %.4f" % weight)
print("The learned bias for the model is %.4f\n" % bias)

plot_the_model(weight, bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)


# make a prediction for house
def predict_house_values(n, feature, label):
    """Predict house values based on feature."""
    batch = training_df[feature][10000:10000 + n]
    predicted_values = my_model.predict_on_batch(x=batch)

    print("feature   label          predicted")
    print("  value   value          value")
    print("          in thousand$   in thousand$")
    print("--------------------------------------")
    for i in range(n):
        print("%5.0f %6.0f %15.0f"% (training_df[feature][10000 + i],
                                     training_df[label][10000 + i],
                                     predicted_values[i][0]) )
predict_house_values(10, my_feature, my_label)
print(training_df.corr())

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
pip install tensorflow 

: 